In [ ]:
import pandas as pd
df = pd.read_excel('all.xlsx')
df.head()

# Spliting

In [ ]:
import pandas as pd
f = "all.xlsx"
ex = pd.ExcelFile(f)
for shit in ex.sheet_names:
    df = pd.read_excel(f,sheet_name=shit)
    out = f"{shit}.xlsx"
    df.to_excel(out,index=False)
    

# pip install optuna

In [ ]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # ignore notifications
import sys
import os


In [ ]:
import pandas as pd
df = pd.read_excel('COM.xlsx')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
for col in df.columns:
    print(f"Value counts for column: {col}")
    print(df[col].value_counts())
    print("-" * 40)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   zone                            1005 non-null   object 
 4   location_hub                    1005 non-null   object 
 5   property_type                   1005 non-null   object 
 6   ownership                       1005 non-null   object 
 7   size_in_sqft                    1005 non-null   int64  
 8   carpet_area_sqft                1005 non-null   object 
 9   private_washroom                1005 non-null   int64  
 10  public_washroom                 1005 non-null   int64  
 11  floor_no                        1005 non-null   object 
 12  total_floors                    10

In [ ]:
df = df.drop('location', axis=1)

In [ ]:
df = df.replace(" ", 0).fillna(0)

In [ ]:
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
print(df['floor_no'].value_counts())

In [ ]:
print(df['amenities_count'].value_counts())

In [ ]:
import pandas as pd
import re

def floor_to_int_list(floor_str):
    floor_str = str(floor_str).lower()
    floor_str = floor_str.replace('floor', '').replace('floors', '').replace(' ', '')
    # Replace ground floor / GF with 0
    floor_str = floor_str.replace('ground', '0').replace('gf', '0')
    # Split multiple floors
    parts = re.split(r'[,]', floor_str)
    floor_numbers = []
    for p in parts:
        try:
            floor_numbers.append(int(p))
        except:
            continue
    # Return as comma-separated string
    if floor_numbers:
        return ','.join(map(str, sorted(floor_numbers)))
    return None

# Apply to your DataFrame
df['floor_no'] = df['floor_no'].apply(floor_to_int_list)

# Show result
print(df[['floor_no']])


In [ ]:
print(df['total_floors'].value_counts())

In [ ]:
import pandas as pd

def total_floors_to_int(floor_str):
    try:
        # Remove 'floors' or 'floor' and spaces, then convert to int
        return int(str(floor_str).lower().replace('floors', '').replace('floor', '').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['total_floors'] = df['total_floors'].apply(total_floors_to_int)




In [ ]:
print(df['total_floors'].value_counts())

In [ ]:
print(df['size_in_sqft'].value_counts())	

In [ ]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['size_in_sqft_int'] = df['size_in_sqft'].apply(size_to_int)

# Check result
print(df[['size_in_sqft', 'size_in_sqft_int']].head(20))


In [ ]:
df['size_in_sqft'] = df['size_in_sqft_int']

In [ ]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['size_in_sqft_int'] = df['size_in_sqft'].apply(size_to_int)

# Check result
print(df[['size_in_sqft', 'size_in_sqft_int']].head(20))


In [ ]:
df = df.drop('size_in_sqft_int', axis=1)

In [ ]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['size_in_sqft_int'] = df['size_in_sqft'].apply(size_to_int)

# Check result
print(df[['size_in_sqft', 'size_in_sqft_int']].head(20))


In [ ]:
df.head()

In [ ]:
print("--- Feature Engineering: One-Hot Encoding Categorical Features ---")
categorical_features = ['City','Area', 'Zone', 'Frurnishing_Status', 'Brokerage', 'Maintenance_Charge', 'Recomened for', 'Muncipla Water Or Bore Water', 'Type of Society', 'Room', 'Type']
for feature in categorical_features:
    if feature in df.columns:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)
        print(f"One-hot encoded '{feature}'.")
    else:
        print(f"Warning: Categorical feature '{feature}' not found in DataFrame for encoding.")


# Transform 'Rent' using natural logarithm to reduce skewness
print("--- Feature Engineering: Log Transformation of 'Rent' ---")
# Add a small constant to Rent to avoid log(0) if any rent value is 0
df['Rent_Price'] = np.log1p(df['Rent_Price']) # Using log1p which is log(1+x)
print("Applied log transformation to 'Rent' column.")

print("\n--- DataFrame after Feature Engineering ---")
print(df.head())
print(df.info())

In [ ]:
# Define features (X) and target (y)
X = df.drop('Rent_Price', axis=1)
y = df['Rent_Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nData split into training (80%) and testing (20%) sets.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
numerical_cols = ['Size_In_Sqft', 'Carpet_Area_Sqft', 'Bedrooms', 'Bathrooms', 'Balcony',
'Number_Of_Amenities', 'Floor_No', 'Total_floors_In_Building',
'Road_Connectivity', 'gated_community', 'gym', 'intercom', 'lift', 'pet_allowed', 'pool',
'security', 'water_supply', 'wifi', 'gas_pipeline', 'sports_facility', 'kids_area',
'power_backup', 'Garden', 'Fire_Support', 'Parking', 'ATM_Near_me', 'Airport_Near_me',
'Bus_Stop__Near_me', 'Hospital_Near_me', 'Mall_Near_me', 'Market_Near_me',
'Metro_Station_Near_me', 'Park_Near_me', 'School_Near_me', 'Property_Age']
# Filter numerical_cols to only include columns actually present in X_train
numerical_cols_present = [col for col in numerical_cols if col in X_train.columns]

# 1. Replace empty strings with NaN
X_train = X_train.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace(r'^\s*$', np.nan, regex=True)

# 2. Convert all numeric columns to float
X_train[numerical_cols_present] = X_train[numerical_cols_present].apply(pd.to_numeric, errors='coerce')
X_test[numerical_cols_present] = X_test[numerical_cols_present].apply(pd.to_numeric, errors='coerce')

# 3. Fill NaN with median for each column
for col in numerical_cols_present:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)

# 4. Scale the numeric columns
scaler = StandardScaler()
X_train[numerical_cols_present] = scaler.fit_transform(X_train[numerical_cols_present])
X_test[numerical_cols_present] = scaler.transform(X_test[numerical_cols_present])

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = RandomForestRegressor(n_estimators=450,n_jobs=-1,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")

In [ ]:
import pandas as pd
import numpy as np

# Get feature importances
importances = rf_model.feature_importances_

# If X_train is a DataFrame
feature_names = X_train.columns




# Convert to percentage
importances_percentage = 100 * importances / importances.sum()

# Create DataFrame of feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

print(feature_importance_df)


In [ ]:
feature_importance_df.to_excel("predicted.xlsx", index=False)

In [ ]:
# --- Saving the trained model and scaler ---
import joblib
print("\n--- Saving Model and Scaler ---")
try:
    joblib.dump(rf_model, 'm3.pkl')
    joblib.dump(scaler, 's3.pkl')
    joblib.dump(X.columns.tolist(), 'f3.pkl') # Save feature names for consistency
    print("Model, scaler, and feature names saved successfully.")
except Exception as e:
    print(f"Error saving model/scaler: {e}")

In [ ]:
import joblib

# Save with compression
joblib.dump(rf_model, "m.pkl", compress=3)
